Using a mix of `partridge` and `gtfstk` with some of my own additions to create daily statistical DataFrames for trips, routes and stops. This will later become a module which we will run on our historical MoT GTFS archive and schedule for nightly runs. 

## Imports and config

In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import partridge as ptg
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import datetime

from gtfs_utils import *

alt.renderers.enable('notebook')
alt.data_transformers.enable('json')

sns.set_style("white")
sns.set_context("talk")
sns.set_palette('Set2', 10)

In [ ]:
LOCAL_GTFS_ZIP_PATH = 'data/gtfs_feeds/2018-03-05.zip' 
LOCAL_TARIFF_PATH = 'data/sample/latest_tariff.zip' 

## Creating a `partridge` feed
We have a util function for getting a `partridge` feed object by date.  

In [ ]:
feed = get_partridge_feed_by_date(LOCAL_GTFS_ZIP_PATH, datetime.date(2018,3,5))
type(feed)

In [ ]:
zones = get_zones_df(LOCAL_TARIFF_PATH)
zones.head()

## Stats

### trip stats
1. calculate using partridge feed
1. add:
    1. stop_code, stop_name
    1. zone

In [8]:
import gtfstk

In [10]:
feed.stop_times.dtypes

trip_id                 object
arrival_time           float64
departure_time         float64
stop_id                 object
stop_sequence            int64
pickup_type            float64
drop_off_type           object
shape_dist_traveled      int64
dtype: object

computing trip_stats corresponding to `gtfstk.compute_trip_stats()` 

In [132]:
feed.trips.direction_id.value_counts()

0    49026
1    41228
Name: direction_id, dtype: int64

check whether we have bidirectionals

In [133]:
feed.trips.groupby('route_id').direction_id.nunique().value_counts()

1    6673
Name: direction_id, dtype: int64

Nope

In [55]:
f = feed.trips
f = (
    f[['route_id', 'trip_id', 'direction_id', 'shape_id']]
    .merge(feed.routes[['route_id', 'route_short_name', 'route_type']])
    .merge(feed.stop_times)
    .merge(feed.stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'stop_code']])
    .merge(zones)
    .sort_values(['trip_id', 'stop_sequence'])
    #.assign(departure_time=lambda x: x['departure_time'].map(
    #    hp.timestr_to_seconds)
    #       )
    )

In [58]:
f.head().T

,2235521,2322805,2241223,2241263,2322737
route_id,9735,9735,9735,9735,9735
trip_id,10096398_040318,10096398_040318,10096398_040318,10096398_040318,10096398_040318
direction_id,1,1,1,1,1
shape_id,88862,88862,88862,88862,88862
route_short_name,70,70,70,70,70
route_type,3,3,3,3,3
arrival_time,75600,75640,75716,75858,76483
departure_time,75600,75640,75716,75858,76483
stop_id,34657,35317,34436,34444,35506
stop_sequence,1,2,3,4,5


In [68]:
geometry_by_stop = gtfstk.build_geometry_by_stop(feed, use_utm=True)


In [69]:
g = f.groupby('trip_id')

In [70]:
from collections import OrderedDict
def my_agg(group):
    d = OrderedDict()
    d['route_id'] = group['route_id'].iat[0]
    d['route_short_name'] = group['route_short_name'].iat[0]
    d['route_type'] = group['route_type'].iat[0]
    d['direction_id'] = group['direction_id'].iat[0]
    d['shape_id'] = group['shape_id'].iat[0]
    d['num_stops'] = group.shape[0]
    d['start_time'] = group['departure_time'].iat[0]
    d['end_time'] = group['departure_time'].iat[-1]
    d['start_stop_id'] = group['stop_id'].iat[0]
    d['end_stop_id'] = group['stop_id'].iat[-1]
    d['start_stop_code'] = group['stop_code'].iat[0]
    d['end_stop_code'] = group['stop_code'].iat[-1]
    d['start_stop_name'] = group['stop_name'].iat[0]
    d['end_stop_name'] = group['stop_name'].iat[-1]
    d['start_zone'] = group['zone_name'].iat[0]
    d['end_zone'] = group['zone_name'].iat[-1]
    dist = geometry_by_stop[d['start_stop_id']].distance(
      geometry_by_stop[d['end_stop_id']])
    d['is_loop'] = int(dist < 400)
    d['duration'] = (d['end_time'] - d['start_time'])/3600
    return pd.Series(d)

In [71]:
h = g.apply(my_agg)

In [72]:
h['distance'] = g.shape_dist_traveled.max()

In [73]:
# Reset index and compute final stats
h = h.reset_index()
h['speed'] = h['distance'] / h['duration'] / 1000
h[['start_time', 'end_time']] = (
  h[['start_time', 'end_time']].applymap(
  lambda x: gtfstk.helpers.timestr_to_seconds(x, inverse=True))
)

In [74]:
h.sort_values(by='speed', ascending=False).head().T

,36646,36647,36648,43945,43944
trip_id,29575729_040318,29575734_040318,29575739_040318,29899643_040318,29899638_040318
route_id,12518,12518,12518,12517,12517
route_short_name,NaN,NaN,NaN,NaN,NaN
route_type,2,2,2,2,2
direction_id,1,1,1,0,0
shape_id,51387,51387,51387,51386,51386
num_stops,2,2,2,2,2
start_time,06:10:00,08:10:00,16:25:00,17:52:00,10:37:00
end_time,06:34:00,08:34:00,16:49:00,18:16:00,11:01:00
start_stop_id,37310,37310,37310,37314,37314


In [76]:
h[h.is_loop==1].sort_values(by='duration', ascending=False).head().T

,29454,29452,29451,29455,29453
trip_id,26534192_040318,26534183_040318,26534179_040318,26534197_040318,26534187_040318
route_id,8067,8067,8067,8067,8067
route_short_name,58,58,58,58,58
route_type,3,3,3,3,3
direction_id,0,0,0,0,0
shape_id,91607,91607,91607,91607,91607
num_stops,56,56,56,56,56
start_time,07:30:00,10:15:00,08:45:00,21:00:00,14:50:00
end_time,09:30:01,12:15:01,10:45:01,23:00:01,16:50:01
start_stop_id,37124,37124,37124,37124,37124


## Route stats

This is mostly taken from `gtfstk.compute_route_stats_base()`, with some additions:

1. first start_stop_id and end_stop_id
1. first start_zone and end_zone
1. first number of stops

In [ ]:
    """
    Compute stats for the given subset of trips stats.

    Parameters
    ----------
    trip_stats_subset : DataFrame
        Subset of the output of :func:`.trips.compute_trip_stats`
    split_directions : boolean
        If ``True``, then separate the stats by trip direction (0 or 1);
        otherwise aggregate trips visiting from both directions. 
        Default: ``False``
    headway_start_time : string
        HH:MM:SS time string indicating the start time for computing
        headway stats
        Default: ``'07:00:00'``
    headway_end_time : string
        HH:MM:SS time string indicating the end time for computing
        headway stats.
        Default: ``'19:00:00'``

    Returns
    -------
    DataFrame
        Columns are

        - ``'route_id'``
        - ``'route_short_name'``
        - ``'agency_id'``
        - ``'agency_name'``
        - ``'route_long_name'``
        - ``'route_type'``
        - ``'direction_id'``: 1/0
        - ``'num_trips'``: number of trips on the route in the subset
        - ``'num_trip_starts'``: number of trips on the route with
          nonnull start times
        - ``'num_trip_ends'``: number of trips on the route with nonnull
          end times that end before 23:59:59
        - ``'is_loop'``: 1 if at least one of the trips on the route has
          its ``is_loop`` field equal to 1; 0 otherwise
        - ``'is_bidirectional'``: 1 if the route has trips in both
          directions; 0 otherwise
        - ``'start_time'``: start time of the earliest trip on the route
        - ``'end_time'``: end time of latest trip on the route
        - ``'max_headway'``: maximum of the durations (in minutes)
          between trip starts on the route between
          ``headway_start_time`` and ``headway_end_time`` on the given
          dates
        - ``'min_headway'``: minimum of the durations (in minutes)
          mentioned above
        - ``'mean_headway'``: mean of the durations (in minutes)
          mentioned above
        - ``'peak_num_trips'``: maximum number of simultaneous trips in
          service (for the given direction, or for both directions when
          ``split_directions==False``)
        - ``'peak_start_time'``: start time of first longest period
          during which the peak number of trips occurs
        - ``'peak_end_time'``: end time of first longest period during
          which the peak number of trips occurs
        - ``'service_duration'``: total of the duration of each trip on
          the route in the given subset of trips; measured in hours
        - ``'service_distance'``: total of the distance traveled by each
          trip on the route in the given subset of trips; measured in
          whatever distance units are present in ``trip_stats_subset``;
          contains all ``np.nan`` entries if ``feed.shapes is None``
        - ``'service_speed'``: service_distance/service_duration;
          measured in distance units per hour
        - ``'mean_trip_distance'``: service_distance/num_trips
        - ``'mean_trip_duration'``: service_duration/num_trips
        - ``'start_stop_id'``: ``start_stop_id`` of the first trip for the route
        - ``'end_stop_id'``: ``end_stop_id`` of the first trip for the route
        - ``'num_stops'``: ``num_stops`` of the first trip for the route
        - ``'start_zone'``: ``start_zone`` of the first trip for the route
        - ``'end_zone'``: ``end_zone`` of the first trip for the route
        
        If not ``split_directions``, then remove the
        direction_id column and compute each route's stats,
        except for headways, using
        its trips running in both directions.
        In this case, (1) compute max headway by taking the max of the
        max headways in both directions; (2) compute mean headway by
        taking the weighted mean of the mean headways in both
        directions.
        
        If ``trip_stats_subset`` is empty, return an empty DataFrame.

    """


In [103]:
headway_start_time='07:00:00'
headway_end_time='19:00:00'
# Convert trip start and end times to seconds to ease calculations below
f = h.copy()
f[['start_time', 'end_time']] = f[['start_time', 'end_time']
  ].applymap(gtfstk.helpers.timestr_to_seconds)

headway_start = gtfstk.helpers.timestr_to_seconds(headway_start_time)
headway_end = gtfstk.helpers.timestr_to_seconds(headway_end_time)

In [114]:
def compute_route_stats(group):
    d = OrderedDict()
    d['route_short_name'] = group['route_short_name'].iat[0]
    d['route_type'] = group['route_type'].iat[0]
    d['num_trips'] = group.shape[0]
    d['num_trip_starts'] = group['start_time'].count()
    d['num_trip_ends'] = group.loc[
      group['end_time'] < 24*3600, 'end_time'].count()
    d['is_loop'] = int(group['is_loop'].any())
    d['is_bidirectional'] = int(group['direction_id'].unique().size > 1)
    d['start_time'] = group['start_time'].min()
    d['end_time'] = group['end_time'].max()

    # Compute headway stats
    headways = np.array([])
    for direction in [0, 1]:
        stimes = group[group['direction_id'] == direction][
          'start_time'].values
        stimes = sorted([stime for stime in stimes
          if headway_start <= stime <= headway_end])
        headways = np.concatenate([headways, np.diff(stimes)])
    if headways.size:
        d['max_headway'] = np.max(headways)/60  # minutes
        d['min_headway'] = np.min(headways)/60  # minutes
        d['mean_headway'] = np.mean(headways)/60  # minutes
    else:
        d['max_headway'] = np.nan
        d['min_headway'] = np.nan
        d['mean_headway'] = np.nan

    # Compute peak num trips
    times = np.unique(group[['start_time', 'end_time']].values)
    counts = [gtfstk.helpers.count_active_trips(group, t) for t in times]
    start, end = gtfstk.helpers.get_peak_indices(times, counts)
    d['peak_num_trips'] = counts[start]
    d['peak_start_time'] = times[start]
    d['peak_end_time'] = times[end]

    d['service_distance'] = group['distance'].sum()
    d['service_duration'] = group['duration'].sum()
    
    # Added by cjer
    d['start_stop_id'] = group['start_stop_id'].iat[0]
    d['end_stop_id'] = group['end_stop_id'].iat[0]
    
    d['num_stops'] = group['num_stops'].iat[0]
    
    d['start_zone'] = group['start_zone'].iat[0]
    d['end_zone'] = group['end_zone'].iat[0]
    
    
    return pd.Series(d)

In [115]:
g = f.groupby('route_id').apply(
compute_route_stats).reset_index()

# Compute a few more stats
g['service_speed'] = g['service_distance']/g['service_duration']
g['mean_trip_distance'] = g['service_distance']/g['num_trips']
g['mean_trip_duration'] = g['service_duration']/g['num_trips']

In [116]:
# Convert route times to time strings
g[['start_time', 'end_time', 'peak_start_time', 'peak_end_time']] =\
    g[['start_time', 'end_time', 'peak_start_time', 'peak_end_time']].\
        applymap(lambda x: gtfstk.helpers.timestr_to_seconds(x, inverse=True))

In [117]:
g['service_speed'] = g.service_speed/1000

In [118]:
g.sort_values(by='num_trips', ascending=False).head(10).T

,814,816,1394,1395,4351,4352,338,339,6549,4481
route_id,11685,11689,13428,13429,2259,2261,10746,10747,9817,2533
route_short_name,1,1,1,1,5,5,1,1,204,66
route_type,3,3,3,3,3,3,0,0,3,3
num_trips,195,186,177,177,156,155,148,148,143,132
num_trip_starts,195,186,177,177,156,155,148,148,143,132
num_trip_ends,192,184,173,173,155,154,147,147,142,129
is_loop,0,0,0,0,0,0,0,0,0,0
is_bidirectional,0,0,0,0,0,0,0,0,0,0
start_time,05:00:00,04:40:00,00:00:00,00:05:00,05:15:00,00:00:00,05:30:00,05:30:00,00:05:00,00:10:00
end_time,24:45:30,24:22:09,24:49:57,24:55:49,24:19:58,24:12:07,24:09:47,24:04:47,24:08:24,24:43:53


### Add stuff
1. agency_id, agency_name
1. route_long_name


In [119]:
g = (g
     .merge(feed.routes[['route_id', 'route_long_name', 'agency_id']], how='left', on='route_id')
     .merge(feed.agency[['agency_id', 'agency_name']], how='left', on='agency_id')
    )

In [120]:
g = g[['route_id', 'route_short_name', 'agency_id', 'agency_name', 'route_long_name', 'route_type', 
           'num_trips', 'num_trip_starts', 'num_trip_ends', 'is_loop', 
           'is_bidirectional', 'start_time', 'end_time', 'max_headway', 'min_headway', 
           'mean_headway', 'peak_num_trips', 'peak_start_time', 'peak_end_time',
           'service_distance', 'service_duration', 'service_speed',
           'mean_trip_distance', 'mean_trip_duration', 'start_stop_id',
           'end_stop_id', 'num_stops', 'start_zone', 'end_zone', 
           ]]

In [135]:
g.sort_values(by='peak_num_trips', ascending=False).head(10).T

,814,1534,816,810,812,1395,4516,1394,6544,5504
route_id,11685,14050,11689,11681,11683,13429,2709,13428,9809,6660
route_short_name,1,1,1,3,3,1,129,1,172,417
agency_id,30,30,30,30,30,5,5,5,5,3
agency_name,דן צפון,דן צפון,דן צפון,דן צפון,דן צפון,דן,דן,דן,דן,אגד
route_long_name,תחנה מרכזית חוף כרמל/רציפים עירוני-חיפה<->מרכז...,מרכזית הקריות-קרית מוצקין<->תחנה מרכזית חוף כר...,מרכזית הקריות-קרית מוצקין<->תחנה מרכזית חוף כר...,הנביאים-חיפה<->מרכזית הקריות-קרית מוצקין-10,מרכזית הקריות-קרית מוצקין<->הנביאים-חיפה-20,מרכז הספורט/דרך בגין-בת ים<->תחנה מרכזית פתח ת...,אבי האסירים-ראשון לציון<->רדינג-תל אביב יפו-20,תחנה מרכזית פתח תקווה/רציפים עירוני-פתח תקווה<...,האורגים-חולון<->רדינג-תל אביב יפו-20,שד.נחל צאלים/נחל חבר-בית שמש<->מסוף אגד/הר חוצ...
route_type,3,3,3,3,3,3,3,3,3,3
num_trips,195,27,186,123,123,177,55,177,111,89
num_trip_starts,195,27,186,123,123,177,55,177,111,89
num_trip_ends,192,27,184,121,121,173,55,173,110,86
is_loop,0,0,0,0,0,0,0,0,0,0


In [138]:
g.sort_values(by='peak_num_trips', ascending=False).head(10).T.to_csv('180305_route_stats_top10_peak_num_trips.csv')

In [125]:
g.is_bidirectional.value_counts()

0    6673
Name: is_bidirectional, dtype: int64

In [140]:
g[g.start_zone==g.end_zone].groupby('start_zone').route_id.nunique()

start_zone
אופקים                        16
אזור בית שמש                  51
אזור חדרה                     19
אזור קריית ארבע                2
אשדוד                         74
אשקלון                        89
באר שבע                       79
בית שאן                       14
גוש דן                       774
גוש עציון                      2
גליל עליון ורמת הגולן        251
הרי ירושלים                   21
השומרון                       44
זכרון                         32
חבל מודיעין                  118
חריש ואדי ערה                134
יוקנעם- טבעון                 78
ירוחם                          2
כרמיאל                       159
נהריה                        125
נצרת                         223
נתיבות שדרות                  60
נתניה                        186
סובב חיפה                    395
סובב ירושלים                 576
סובב כנרת ודרום רמת הגולן    103
עומר חורה                      2
עכו                           64
עפולה                         54
ערד דימונה                    59

In [141]:
g.shape

(6673, 29)

### add date

In [ ]:
g['date'] = 

## What's next

TODO

1. add date
1. add split_directions
1. add time between stops - max, min, mean (using delta)
1. integrate with custom day cutoff
1. add day and night headways and num_trips (maybe noon also)
1. create a function for converting this to a timeseries good for pandas
1. put this all back into proper documented functions
1. write tests
